# LISTA 1

## GLOBAL SETUP

In [3]:
import numpy as np
import os
import statistics
from tqdm.notebook import tqdm
import re
from sys import getsizeof
import sys
import copy
import nltk
import collections
import random
import itertools

## ZADANIE 1

In [4]:
def Succ1Grams():
    data = set()
    
    #num_lines = sum(1 for line in open('poleval_2grams.txt'))
    num_lines = 59134224
    lines_read = 0
    with open("poleval_2grams.txt", "r") as f:
        with tqdm(total=num_lines) as pbar:
            line = f.readline()
            while line:
                if lines_read > 10000:
                    pbar.update(lines_read)
                    lines_read = 0
                    pbar.set_description(f"memory used {sys.getsizeof(data)/1024/1024} MB")
                lines_read += 1
                wc, wl, wp = line.replace("\n", "").split(" ")
                data.add(wl)
                data.add(wp)

                #if len(t) < 50:
                #    data.append(t)
                line = f.readline()
                if len(data) > 10000000:
                    break
    return data

singletons = Succ1Grams()
maxlen = max(len(s) for s in singletons)
print(f"read {len(singletons)}, longest one is {maxlen}")
maxlen = 25


read 3591116, longest one is 125


In [5]:
def OwnMatch(w):
    graph = [[] for _ in w]
    for slit in range(len(w)):
        for forward_succ in range(5):
            taken = w[slit:slit+forward_succ+1]
            if len(taken) == forward_succ+1:
                if taken in singletons:
                    graph[slit].append(slit + forward_succ + 1)

    #print(f"neighbor matrix")
    #display(graph)

    dp = [(0, None) for _ in range(len(w)+1)]
    for i in range(len(w)-1):
        for jt in graph[i]:
            if dp[jt][0] < (dp[i][0] + (i-jt)**2):
                dp[jt] = (dp[i][0] + (i-jt)**2, i)

    #display(dp)

    baked = []
    ls = dp[-1][1]
    fs = len(w)
    while ls != None:
        baked.append(w[ls:fs])
        fs = ls
        ls = dp[ls][1]
        
    baked.reverse()
    
    return baked 

In [6]:
def MaxMatch(w, verbose = False):
    wp = w
    tokenized = []
    succ = 1
    best = (0, "reeee")
    
    while len(w) > 0:
        try_ = w[:succ]
        if try_ in singletons:
            if verbose:
                print(f"found candidate {try_}")
            best = (succ, try_) 

        succ += 1
        
        if succ > maxlen or try_ == w:
            if best[0] == 0:
                raise Exception(f"could not match {w[:succ]} in {wp}")
            else:
                if verbose:
                    print(f"longest match was {best[1]}")
                tokenized.append(best[1])
                if verbose:
                    print(f"transforming {w} -> {w[best[0]:]}")
                w = w[best[0]:]
                succ = 1
                best = (0, "reeee")
    return tokenized

In [7]:
# implementation stolen from https://blog.paperspace.com/implementing-levenshtein-distance-word-autocomplete-autocorrect/
# since jupyter considered that importing
# python-levenstein is on the same level as breaking the geneva
# convention

def levenshteinDistanceDP(token1, token2):
    distances = np.zeros((len(token1) + 1, len(token2) + 1))

    for t1 in range(len(token1) + 1):
        distances[t1][0] = t1

    for t2 in range(len(token2) + 1):
        distances[0][t2] = t2
        
    a = 0
    b = 0
    c = 0
    
    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            if (token1[t1-1] == token2[t2-1]):
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - 1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]
                
                if (a <= b and a <= c):
                    distances[t1][t2] = a + 1
                elif (b <= a and b <= c):
                    distances[t1][t2] = b + 1
                else:
                    distances[t1][t2] = c + 1

    return distances[len(token1)][len(token2)]

In [8]:
def perform_test(compare_string):
    digested = compare_string.replace(" ", "")
    p1 = " ".join(MaxMatch(digested))
    p2 = " ".join(OwnMatch(digested))
    s1 = levenshteinDistanceDP(compare_string, p1)
    s2 = levenshteinDistanceDP(compare_string, p2)
    #print(compare_string)
    #print(f"MaxMatch returned {p1}, score {s1}")
    #print(f"OwnCrap returned {p2}, score {s2}")
    return (s1, s2)

In [9]:
poem = '''Jedzie Wojtuś popod lasem, skrajem dąbrowy,
Wiezie se do domu laser, laser gazowy.
Pytali się go kolesie, po co to nabył?
- A bo właśnie stał w GS-ie, więc kupiłem go Teresie.
Dyć coś zawsze przywieźć chce się dla swej baby!
Jedzie Wojtuś popod lasem, po twardym dukcie.
Wiezie se do domu laser, czyta instrukcję.
A instrukcja jest ciekawa: Włączyć do prądu,
To ten laser wszystko skrawa, bez różnicy, stal czy trawa
I w ogóle jest zabawa prawie bez swądu.
Jedzie Wojtuś popod lasem, woła: Wio, wiśta!
Wiezie se do domu laser, tak se rozmyśla:
Jak w tym dojdę do biegłości, będzie wygodnie,
Jest tu paru wrednych gości, przyceluję se w skrytości
I padalcom z odległości podpalę spodnie!
Jedzie Wojtuś popod lasem, rad, że o rany
Wiezie se do domu laser, układa plany:
Niech spróbuje na rowerze jeździć Kaczmarski,
Zaraz w dętkę mu przymierzę, i kartofle się obierze,
I wywierci dziury w serze, by był śwajcarski!
Jedzie Wojtuś popod lasem, skręcił przy POM-ie.
Wiezie se do domu laser schowany w słomie.
Na podwórku cielę bryka, kaczki na stawie...
Wdziera, wdziera się technika coraz częściej w dom rolnika,
Jeszcze to nie Ameryka.... Ale już prawie!!! '''

tMM, tMC = [], []
for l in tqdm(poem.split("\n")):
    retMM, retMC = perform_test(l.lower().replace(",", ""))
    tMM.append(retMM)
    tMC.append(retMC)
    
print("MaxMatch:")
print(f"Mean: {statistics.mean(tMM)}")
print(f"Stdev: {statistics.stdev(tMM)}")
print(f"max: {max(tMM)}")
print(f"min: {min(tMM)}")

print("MyCrap:")
print(f"Mean: {statistics.mean(tMC)}")
print(f"Stdev: {statistics.stdev(tMC)}")
print(f"max: {max(tMC)}")
print(f"min: {min(tMC)}")


#MaxMatch:
#Mean: 5.96
#Stdev: 4.6141087980237305
#max: 17.0
#min: 0.0
#MyCrap:
#Mean: 13.68
#Stdev: 12.294036494712927
#max: 48.0
#min: 4.0


MaxMatch:
Mean: 5.6
Stdev: 3.547299442298794
max: 12.0
min: 0.0
MyCrap:
Mean: 17.8
Stdev: 16.91646141090585
max: 48.0
min: 3.0


## ZADANIE 2 i 3


In [10]:
def calculate_bigrams():
    ins = collections.defaultdict(lambda: 0)
    bigrams = collections.defaultdict(lambda: copy.deepcopy(ins))
    num_lines = 59134224
    lines_read = 0
    with open("poleval_2grams.txt", "r") as f:
        with tqdm(total=num_lines) as pbar:
            line = f.readline()
            while line:
                if lines_read > 10000:
                    pbar.update(lines_read)
                    lines_read = 0
                    pbar.set_description(f"memory used {sys.getsizeof(bigrams)/1024/1024} MB")
                lines_read += 1
                wc, wl, wp = line.replace("\n", "").split(" ")
                bigrams[wl][wp] = int(wc)

                line = f.readline()
                if len(bigrams) > 10000000:
                    break
    return bigrams
            
bigrams = calculate_bigrams()
bigrams['dupa']

defaultdict(<function __main__.calculate_bigrams.<locals>.<lambda>()>,
            {'zresztą': 1,
             'fly': 2,
             'bita': 1,
             'będzie': 1,
             '.': 27,
             ',': 39,
             '-': 4,
             '(': 2,
             'w': 11,
             'z': 16,
             'o': 1,
             'i': 10,
             '2001r': 1,
             'puszyste': 2,
             'tam': 1,
             'starsze': 2,
             '....': 1,
             'panie': 1,
             '2.': 1,
             'naprawde': 1,
             'obok': 1,
             'urosnie': 1,
             'blada': 20,
             '!': 2,
             ')': 2,
             'a': 4,
             'zapragnie': 1,
             'niezły': 1,
             'imo': 1,
             'podobne': 1,
             'ich': 1,
             'latynoski': 5,
             'mnie': 1,
             'pornosy': 1,
             'nic': 1,
             'nie': 7,
             'użyłam': 1,
             'lecz': 1,
          

In [11]:
import time
def generate_story(max_len, starting, distribution):
    if max_len == 0:
        max_len = 1000000
    if starting == None:
        starting = "<BOS>"
        
    while(max_len > 0):
        print(starting, end = ' ')
        max_len -= 1
        candidates = bigrams[starting]
        
        if distribution == "random":
            candidates = [i[0] for i in candidates.items()]
            if len(candidates) == 0:
                return
            starting = random.choice(candidates) 
            
        if distribution == "weight":
            if len([i[0] for i in candidates.items()]) == 0:
                return
            
            all_ents = sum([i[1] for i in candidates.items()])
            chosen = random.randint(0, all_ents)
            for cit in candidates.items():
                chosen -= cit[1]
                if chosen <= 0:
                    starting = cit[0]
                    break
                    
            

In [12]:
for i in range(10):
    generate_story(0, "papież", "weight")
    print("\n=======")

papież jan kobuszewski , gdzie prowadził intensywny ogień artyleryjski c7p zatwierdzono wnioski : a 3 art. 12 lutego przyniosła w lizbonie . <EOS> 
papież benedykt xvi wieku sony ericsson c702 , podzespoły komputerowe itp. ) . <EOS> 
papież bardzo wielu pracowników , czytelnik gazet , grając m.in. w takim określeniem zasad prowadzenia własnej winy . <EOS> 
papież jan draus podał w stanach zjednoczonych szacuje , hala globus w wyniku modernizacji i będzie jedynie niedokończona książka została ona realizowana zgodnie z obozu transporty zboża , wyróżniających się więc rząd rewolucyjny pomysł jednak zobowiązany jest policjantem , pisał o terenach cmentarzy braci jej ubytku wpływów w art. 226 posłów jest sens demokracji nie wraca do francji uciekło z tym etapie opisujemy plan na rynek pracy chronionej . <EOS> 
papież umartwiał swoje spory . <EOS> 
papież klemens murańka i cięższe od postawionych w mojej ocenie sądu rodzinnego z odsetek , którą narażonych korzystać z o.o. koncesji na indywid

## ZADANIE 4

In [13]:
def calculate_sufix(suflen):
    ins = collections.defaultdict(lambda: 0)
    sufgrams = collections.defaultdict(lambda: copy.deepcopy(ins))
    procd = 0
    pbar = tqdm(total=len(bigrams.keys()))
    
    for w1 in bigrams.keys():
        procd += 1

        if procd > 1000:
            procd = 0
            pbar.update(1000)
            pbar.set_description(f"memory used {str(sys.getsizeof(sufgrams)/1024/1024)[:5]} MB")

        if len(w1) > suflen:
            pref_sufix = str(w1)[-suflen:]
            for w2 in bigrams[w1].keys():
                if len(str(w2)) > suflen:
                    suf_sufix = str(w2)[-suflen:]
                    sufgrams[pref_sufix][suf_sufix] += bigrams[w1][w2]
                    
    # normalize ...
    biggest = 0
    for w1 in sufgrams.keys():
        for w2 in sufgrams[w1].keys():
            biggest = max(biggest, sufgrams[w1][w2])
    print(f"maximum score was {biggest}")
    for w1 in sufgrams.keys():
        for w2 in sufgrams[w1].keys():
            sufgrams[w1][w2] = float(sufgrams[w1][w2])/float(biggest)
    
    pbar.close()
    return sufgrams

In [14]:
bigram_suflist = [calculate_sufix(i) for i in range(1, 6)]

maximum score was 7795541



maximum score was 2372865



maximum score was 1022463



maximum score was 269347



maximum score was 128690



In [ ]:
def succ_3grams():
    ins = collections.defaultdict(lambda: 0)
    insd = collections.defaultdict(lambda: copy.deepcopy(ins))
    trigrams = collections.defaultdict(lambda: copy.deepcopy(insd))
    
    num_lines = 179473348
    lines_read = 0
    mv = 0
    with open("poleval_3grams.txt", "r") as f:
        with tqdm(total=num_lines) as pbar:
            line = f.readline()
            while line:
                if lines_read > 10000:
                    pbar.update(lines_read)
                    lines_read = 0
                    pbar.set_description(f"memory used {sys.getsizeof(trigrams)/1024/1024} MB")

                lines_read += 1
                try:
                    wc, w1, w2, w3 = line.replace("\n", "").split(" ")
                    trigrams[w1][w2][w3] += int(wc)
                    mv = max(mv, trigrams[w1][w2][w3])
                except Exception:
                    pass
                line = f.readline()
 
    return (bigrams, mv)
            
trigrams, max_c = succ_3grams()
print(f"normalize by {max_c}")
trigrams['dupa']

In [ ]:
def check_for_bigram(wa, wb):
    return bigrams[wa][wb]

def check_for_bisuf(wa, wb):
    best = 0
    it = 5
    if len(wa) > it and len(wb) > it:
        sufa = wa[-it:]
        sufb = wa[-it:]
        best = max(best, bigram_suflist[it-1][sufa][sufb])
    return best


def permutation_analizer(sentence):
    results = []
    for pp in itertools.permutations(sentence):
        p = ["<BOS>"] + list(pp) + ["<EOS>"]
        
        bs = sum(check_for_bigram(p[pit], p[pit+1]) for pit in range(len(p) - 1))
        bss = sum(check_for_bisuf(p[pit], p[pit+1]) for pit in range(len(p) - 1))
        
        results.append((-bss, p))
    
    res = sorted(results)
    for r in res[:10]:
        print(r)
            
permutation_analizer("wczoraj wieczorem spotkałem pewną piękną kobietę".split(" "))
#permutation_analizer(["judyta", "dała", "wczoraj", "stefanowi", "czekoladki"])